## A notebook that runs UNet prediction in batches

by only loading the model once to the gpu prediction is much faster 

In [ ]:
#import packages

import os
from datetime import datetime

import numpy as np
import os 
import argparse
import logging

import sys

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image


sys.path.insert(0,"./Pytorch-UNet/")
from utils.data_loading import BasicDataset
from unet import UNet
from utils.utils import plot_img_and_mask



In [ ]:
# add relevant functions:

def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()
    img = torch.from_numpy(BasicDataset.preprocess(None, full_img, scale_factor, is_mask=False))
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img).cpu()
        output = F.interpolate(output, (full_img.size[1], full_img.size[0]), mode='bilinear')
        if net.n_classes > 1:
            mask = output.argmax(dim=1)
        else:
            mask = torch.sigmoid(output) > out_threshold

    return mask[0].long().squeeze().numpy()



def mask_to_image(mask: np.ndarray, mask_values):
    if isinstance(mask_values[0], list):
        out = np.zeros((mask.shape[-2], mask.shape[-1], len(mask_values[0])), dtype=np.uint8)
    elif mask_values == [0, 1]:
        out = np.zeros((mask.shape[-2], mask.shape[-1]), dtype=bool)
    else:
        out = np.zeros((mask.shape[-2], mask.shape[-1]), dtype=np.uint8)

    if mask.ndim == 3:
        mask = np.argmax(mask, axis=0)

    for i, v in enumerate(mask_values):
        out[mask == i] = v

    return Image.fromarray(out)


In [ ]:
#initialize cuda 

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use GPU with index 1


In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the default CUDA device
    device = torch.device("cuda")

    # Query the available memory
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    free_memory = total_memory - allocated_memory

    print(f"Total GPU Memory: {total_memory / 1024**2:.2f} MB")
    print(f"Allocated GPU Memory: {allocated_memory / 1024**2:.2f} MB")
    print(f"Free GPU Memory: {free_memory / 1024**2:.2f} MB")
else:
    print("CUDA is not available.")



Specify input paramaters 

In [ ]:
image_dir = '/home/lmeyers/paintDetect/images/testing/'
model = '/home/lmeyers/paintDetect/wandb/latest-run/files/model.pth'

The below code will create a new timestamped directory in the mask folder where predictions will be saved. 
It wil then load the model to the gpu and call each file in the image_dir and save with a .pred.jpg 

In [ ]:

current_date = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
directory_name = f"/home/lmeyers/paintDetect/masks/predict_{current_date}"
os.mkdir(directory_name)

#image_dir = "/home/lmeyers/paintDetect/images/testing/"


in_files = []
out_files = []

dir_list = os.walk(image_dir)
for root, dirs, files in dir_list:
    for f in files: 
        #print(root+f)
        in_files.append(root+f)
        out_files.append(f"/home/lmeyers/paintDetect/masks/predict_{current_date}/"+f[:-4]+'.pred.jpg')


logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


model = '/home/lmeyers/paintDetect/wandb/latest-run/files/model.pth'

net = UNet(n_channels=3, n_classes=2, bilinear=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Loading model {model}')
logging.info(f'Using device {device}')

net.to(device=device)
state_dict = torch.load(model, map_location=device)
mask_values = state_dict.pop('mask_values', [0, 1])
net.load_state_dict(state_dict)

logging.info('Model loaded!')

for i, filename in enumerate(in_files):
    logging.info(f'Predicting image {filename} ...')
    img = Image.open(filename)

    mask = predict_img(net=net,
                        full_img=img,
                        scale_factor=1,
                        out_threshold=.5,
                        device=device)

   
    out_filename = out_files[i]
    result = mask_to_image(mask, mask_values)
    result.save(out_filename)
    logging.info(f'Mask saved to {out_filename}')

In [ ]:
#making prediction on a folder a function for easy calling 

def predict_on_folder(in_files,out_files,model_path):
    '''
    a functionized version of the cell above for predicting using a UNet on a set of images
    
    Parameters:
    in_files(list): a list of image paths to be predicted on
    out_files(list): the paths of the files to be saved from prediction 
    model_path(str): the path to the pytorch model to use for inference

    Returns:
    saves outfiles

    '''
    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


    model = model_path

    net = UNet(n_channels=3, n_classes=2, bilinear=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Loading model {model}')
    logging.info(f'Using device {device}')

    net.to(device=device)
    state_dict = torch.load(model, map_location=device)
    mask_values = state_dict.pop('mask_values', [0, 1])
    net.load_state_dict(state_dict)

    logging.info('Model loaded!')

    for i, filename in enumerate(in_files):
        logging.info(f'Predicting image {filename} ...')
        img = Image.open(filename)

        mask = predict_img(net=net,
                            full_img=img,
                            scale_factor=1,
                            out_threshold=.5,
                            device=device)

    
        out_filename = out_files[i]
        result = mask_to_image(mask, mask_values)
        result.save(out_filename)
        logging.info(f'Mask saved to {out_filename}')

## Making a new dataset with just masks of paint

This will be a implementation of batch predict run on nested folders with paint masking looped in 

In [ ]:
import cv2
def crop_by_mask(image,mask,output_path):
    """
    takes in an image, and a corresponding binary mask and saves new file of image 
    values only on pixels where mask was positive. 

    Parameters:
    image (str): a string of path to image.
    mask (str): a string og path to mask 

    Returns:
    saves file to image_path/image_name.paint_only.png
    """
    img = cv2.imread(image)
    mask = cv2.imread(mask)
    output_path = output_path
    image_name = os.path.basename(image)

    # Convert the image to grayscale
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # Apply a threshold to create a binary image
    ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the mask
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        print("No contours found in the mask.")
        return

    # Find the bounding box of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image based on the bounding box
    #cropped_image = img[y:y+h, x:x+w]

    # Create a mask of the cropped region
    #cropped_mask = mask[y:y+h, x:x+w]

    # Apply the mask to the cropped image
    #result = cv2.bitwise_and(cropped_image, cropped_image, mask=cropped_mask)
    result = cv2.bitwise_and(img,img,mask=binary)

    # Save the result to the output path
    cv2.imwrite(output_path, result)
    #cv2.imshow('crop',result)
    #cv2.waitKey(5000)
    #cv2.destroyAllWindows()
    print(f"Cropped image saved to: {output_path}")



In [8]:
#do predict for each folder in a dir and save to same loc as image path

image_dir = '/home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/'
model = '/home/lqmeyers/paintDetect/models/thorax_segmentation_best_run_20230526_003438.pth'


os.mkdir(image_dir+"Masks")


dir_list = os.walk(image_dir)
#print(dir_list)
for root, dirs, files in dir_list:
    dirs.sort()
    for dir in dirs:
        #print(dir)
        if dir != "Masks" and dir != "flat":
            files_list = os.walk(root+dir)
            print(root+dir)
            os.mkdir(image_dir+"Masks"+r"/"+dir)
            in_files = []
            out_files = []
            for r, d, f in files_list:
                for fs in f:
                    in_files.append(root+dir+r"/"+fs)
                    out_files.append(image_dir+"Masks/"+dir+r"/"+fs[:-4]+".paint_mask.jpg")
            #print(in_files)
            predict_on_folder(in_files,out_files,"/home/lqmeyers/paintDetect/models/paint_segmentation_only_best_run_20230525_113551.pth")
            print("masks predicted for"+root+dir)
            for fs in f:
                crop_by_mask(root+dir+r"/"+fs,image_dir+"Masks/"+dir+r"/"+fs[:-4]+".paint_mask.jpg",image_dir+"Masks/"+dir+r"/"+fs[:-4]+".paint_mask.jpg")
            
            
            


INFO: Loading model /home/lqmeyers/paintDetect/models/paint_segmentation_only_best_run_20230525_113551.pth
INFO: Using device cuda


/home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#01BigBlueUpThor


INFO: Model loaded!
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#01BigBlueUpThor/f1.1x2022_06_22.mp4.track000037.frame004487.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f1.1x2022_06_22.mp4.track000037.frame004487.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#01BigBlueUpThor/f3.1x2022_06_22.mp4.track000011.frame000497.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f3.1x2022_06_22.mp4.track000011.frame000497.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#01BigBlueUpThor/f1.1x2022_06_22.mp4.track000008.frame002737.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f1.1x2022_06_22.mp4.track000008.frame002737.paint_mask.jpg
INFO: Predicti

masks predicted for/home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#01BigBlueUpThor
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f1.1x2022_06_22.mp4.track000037.frame004487.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f3.1x2022_06_22.mp4.track000011.frame000497.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f1.1x2022_06_22.mp4.track000008.frame002737.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f1.2x2022_06_22.mp4.track000003.frame001113.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor/f1.2x2022_06_22.mp4.track000001.frame000128.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files

INFO: Loading model /home/lqmeyers/paintDetect/models/paint_segmentation_only_best_run_20230525_113551.pth
INFO: Using device cuda
INFO: Model loaded!
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#02WhitAbd/f1.1x2022_06_22.mp4.track000024.frame003658.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#02WhitAbd/f1.1x2022_06_22.mp4.track000024.frame003658.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#02WhitAbd/f1.2x2022_06_22.mp4.track000066.frame007437.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#02WhitAbd/f1.2x2022_06_22.mp4.track000066.frame007437.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#02WhitAbd/f1.1x2022_06_22.mp4.track000091.frame015105.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/fileso

...

INFO: Loading model /home/lqmeyers/paintDetect/models/paint_segmentation_only_best_run_20230525_113551.pth
INFO: Using device cuda
INFO: Model loaded!
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000368.frame025337.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000368.frame025337.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000463.frame035374.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000463.frame035374.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000575.frame037265.jpg ...
INFO: Mask saved to /home/lqmeyers

masks predicted for/home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#30YellowRightBigThor
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000368.frame025337.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000463.frame035374.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000575.frame037265.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000395.frame026828.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#30YellowRightBigThor/f2x2022_07_22.mp4.track000306.frame022217.paint_mask.jpg
Cropped image saved to: /home/

INFO: Loading model /home/lqmeyers/paintDetect/models/paint_segmentation_only_best_run_20230525_113551.pth
INFO: Using device cuda
INFO: Model loaded!
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000001.frame000321.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000001.frame000321.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000001.frame000304.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000001.frame000304.paint_mask.jpg
INFO: Predicting image /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000018.frame000241.jpg ...
INFO: Mask saved to /home/lqmeyers/SLEAP_files/Bee_imgs/flo

masks predicted for/home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/ID#31BigRedSplotch
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000001.frame000321.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000001.frame000304.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000018.frame000241.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000063.frame000967.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#31BigRedSplotch/f7x2022_06_20.mp4.track000031.frame000402.paint_mask.jpg
Cropped image saved to: /home/lqmeyers/SLEAP_files/Bee_imgs/

FileExistsError: [Errno 17] File exists: '/home/lqmeyers/SLEAP_files/Bee_imgs/flowerpatch_imgs/filesort/Masks/ID#01BigBlueUpThor'

In [7]:

# Clear GPU memory
torch.cuda.empty_cache()